## TODO

1. find out how to fit the tokenized thing in the model
2. put the code in a python script so that we can run it on the competition day
3. how to read the answer.evaluate

# Insert data and tokenize it

In [1]:
#To run this notebook, you need to have the folder "AI_Course_Final" (downloaded from Kaggle) 
#in the parent folder of the git-repository-folder

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import copy
import os
from functools import reduce
import numpy as np
import h5py

Using TensorFlow backend.
/Users/fridahammarberg/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
current = os.getcwd()
print(current) # displays our current directory
parent = os.path.abspath(os.path.join(current, os.pardir)) 
print(parent) # displays the parent directory
path = parent + "/AI_Course_Final/CBTest/data/cbt_test.txt"
print(path) 
os.path.exists(path)
# Check if the wanted path exist. Must be True!!
test = pd.read_csv(path, sep='\n')

/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/intelligent_computing_project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/AI_Course_Final/CBTest/data/cbt_test.txt


In [5]:
path_CN_test = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_test_2500ex.txt"
path_CN_train = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_train.txt"
path_CN_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_valid_2000ex.txt"

path_NE_test = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_test_2500ex.txt"
path_NE_train = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_train.txt"
path_NE_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_valid_2000ex.txt"

path_P_test = parent + "/AI_Course_Final/CBTest/data/cbtest_P_test_2500ex.txt"
path_P_train = parent + "/AI_Course_Final/CBTest/data/cbtest_P_train.txt"
path_P_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_P_valid_2000ex.txt"

path_V_test = parent + "/AI_Course_Final/CBTest/data/cbtest_V_test_2500ex.txt"
path_V_train = parent + "/AI_Course_Final/CBTest/data/cbtest_V_train.txt"
path_V_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_V_valid_2000ex.txt"

In [6]:
pd.set_option('display.max_colwidth', -1)

CN_test = pd.read_csv(path_CN_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
CN_train = pd.read_csv(path_CN_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
CN_valid = pd.read_csv(path_CN_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

NE_test = pd.read_csv(path_NE_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
NE_train = pd.read_csv(path_NE_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
NE_valid = pd.read_csv(path_NE_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

P_test = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
P_train = pd.read_csv(path_P_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
P_valid = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

V_test = pd.read_csv(path_V_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
V_train = pd.read_csv(path_V_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
V_valid = pd.read_csv(path_V_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

In [7]:
test_df = pd.concat([CN_test, NE_test, P_test, V_test])

train_df = pd.concat([CN_train, NE_train, P_train, V_train])

valid_df = pd.concat([CN_valid, NE_valid, P_valid, V_valid])

In [8]:
test_data = test_df[['sentence']].copy()
train_data = train_df[['sentence']].copy()
valid_data = valid_df[['sentence']].copy()

In [9]:
# delete the nbrs in the beginning of each sentence
test_data['sentence'] = test_data['sentence'].map(lambda x: x.lstrip('1234567890'))
train_data['sentence'] = train_data['sentence'].map(lambda x: x.lstrip('1234567890'))
valid_data['sentence'] = valid_data['sentence'].map(lambda x: x.lstrip('1234567890'))

In [10]:
test_answer = test_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_answer = train_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
valid_answer = valid_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [11]:
test_option = test_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_option = train_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
valid_option = valid_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [12]:
test_option_list = []
k = 0
for x in test_option.iterrows():
    string_opt = test_option.values[k][0].replace("|", " ")
    test_option_list.append(string_opt)
    k += 1
    
test_option_df = pd.DataFrame(test_option_list, columns=['options'])


train_option_list = []
k = 0
for x in train_option.iterrows():
    string_opt = train_option.values[k][0].replace("|", " ")
    train_option_list.append(string_opt)
    k += 1
    
train_option_df = pd.DataFrame(train_option_list, columns=['options'])


valid_option_list = []
k = 0
for x in valid_option.iterrows():
    string_opt = valid_option.values[k][0].replace("|", " ")
    valid_option_list.append(string_opt)
    k += 1
    
valid_option_df = pd.DataFrame(valid_option_list, columns=['options'])

In [13]:
#print(len(test_data))
#print(len(train_data))
#print(len(valid_data))

#print(len(test_answer))
#print(len(train_answer))
#print(len(valid_answer))

#print(len(test_option_df))
#print(len(train_option_df))
#print(len(valid_option_df))


#***


#print(len(test_data_list))
#print(len(train_data_list))
#print(len(valid_data_list))

#print(len(test_answer_list))
#print(len(train_answer_list))
#print(len(valid_answer_list))

#print(len(test_option_list))
#print(len(train_option_list))
#print(len(valid_option_list))

In [14]:
test_data_list = test_data['sentence'].tolist()
train_data_list = train_data['sentence'].tolist()
valid_data_list = valid_data['sentence'].tolist()

test_answer_list = test_answer['answer'].tolist()
train_answer_list = train_answer['answer'].tolist()
valid_answer_list = valid_answer['answer'].tolist()

In [15]:
n = 10
new_test_data_list = copy.deepcopy(test_data_list[:21*2*n])
new_test_answer_list = copy.deepcopy(test_answer_list[:2*n])
new_test_option_list = copy.deepcopy(test_option_list[:2*n])

In [16]:
new_train_data_list = copy.deepcopy(train_data_list[:21*8*n])
new_train_answer_list = copy.deepcopy(train_answer_list[:8*n])
new_train_option_list = copy.deepcopy(train_option_list[:8*n])

In [17]:
print('train:')
print(len(new_train_data_list)/21)
print(len(new_train_answer_list))
print(len(new_train_option_list))
print('test:')
print(len(new_test_data_list)/21)
print(len(new_test_answer_list ))
print(len(new_test_option_list))

train:
80.0
80
80
test:
20.0
20
20


In [18]:
tokenizer = Tokenizer()

In [19]:
#tokenizer.fit_on_texts(test_data_list)
#tokenizer.fit_on_texts(train_data_list)
#tokenizer.fit_on_texts(valid_data_list)

In [20]:
#new_test_data_list[20+21*5] #use this one to test so the result below is correct!

In [21]:
#new_test_data_list/test_data_list to small_test_data_list

t = 0
small_list = []
small_test_data_list = []
for u in new_test_data_list:
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_test_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1

small_test_data_list.append(reduce(lambda a, b: a + b, small_list))
#small_test_data_list[5]

#***********************************

#new_train_data_list/train_data_list to small_train_data_list

t = 0
small_list = []
small_train_data_list = []
for u in new_train_data_list:
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1
small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
#***********************************

In [22]:
print(len(small_test_data_list))
print(len(small_train_data_list))

20
80


In [23]:
#tokenizer.fit_on_texts(new_train_data_list)
tokenizer.fit_on_texts(small_train_data_list)

In [24]:
train_data_matrix = tokenizer.texts_to_matrix(small_train_data_list)
train_answer_matrix = tokenizer.texts_to_matrix(new_train_answer_list)
train_options_matrix = tokenizer.texts_to_matrix(new_train_option_list)

In [25]:
test_data_matrix = tokenizer.texts_to_matrix(small_test_data_list)
test_answer_matrix = tokenizer.texts_to_matrix(new_test_answer_list)
test_options_matrix = tokenizer.texts_to_matrix(new_test_option_list)

In [26]:
test_data_matrix

array([[ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.]])

In [27]:
type(test_data_matrix[0][0])

numpy.float64

In [28]:
#train_data_matrix = tokenizer.texts_to_matrix(train_data_list)
#train_options_matrix = tokenizer.texts_to_matrix(train_option_list)
#train_answer_matrix = tokenizer.texts_to_matrix(train_answer_list)

In [29]:
#test_data_matrix = tokenizer.texts_to_matrix(test_data_list)
#test_options_matrix = tokenizer.texts_to_matrix(test_option_list)
#test_answer_matrix = tokenizer.texts_to_matrix(test_answer_list)

In [30]:
#valid_data_matrix = tokenizer.texts_to_matrix(valid_data_list)
#valid_options_matrix = tokenizer.texts_to_matrix(valid_option_list)
#valid_answer_matrix = tokenizer.texts_to_matrix(valid_answer_list)

# Use Pickle for store the matrices...

https://pythontips.com/2013/08/02/what-is-pickle-in-python/

In [31]:
best_train_answer_matrix = np.expand_dims(train_answer_matrix, -1)
best_test_answer_matrix = np.expand_dims(test_answer_matrix, -1)

# Creating model

In [32]:
from keras.layers import Dense, Merge, Dropout, Permute, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import h5py

# Contants
VOCABULARY_SIZE = 729 #53628 ## estimate from the goldilocks paper
HIDDEN_LAYER_SIZE = 64  ## 10 neurons for 10 options
LSTM_OUTPUT_SIZE=10 #TODO: change LSTM SIZE
BATCH_SIZE = 32
EPOCHS_SIZE = 5

#Initial Embedding matrices A, B for story and options
#Each sentence is taken as an n-dimensional vector of float64, with n=VOCABULARY-SIZE
#memory_encoder encodes lines 1-21 from story, answer encoder
memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 21
memory_encoder.add(Dropout(0.3))

option_encoder = Sequential()
option_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 1
option_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
match = Sequential()
match.add(Merge([memory_encoder, option_encoder], mode="dot", dot_axes=[2, 2]))

#Story embedded with matrix C to produce output vectors
output_memory_encoder = Sequential()
output_memory_encoder.add(Embedding(input_dim=VOCABULARY_SIZE,output_dim = 729,input_length = 729)) #input length = 21
output_memory_encoder.add(Dropout(0.3))

# Response vector, obtained by summing output vectors
# Output dim: (None, 10 options, 21)
response_vector = Sequential()
response_vector.add(Merge([match, output_memory_encoder], mode="sum"))
response_vector.add(Permute((2, 1)))

#Final embedding for prediction, weighting matrix W
#combine response and question vectors and do backpropagation
prediction = Sequential()
prediction.add(Merge([response_vector, option_encoder], mode="concat", concat_axis=-1))
#prediction.add(LSTM(LSTM_OUTPUT_SIZE)) #for soft memory attention mechanism
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation("softmax")) #for softmax 

prediction.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

#prediction.fit([Xs_train, Xq_train, Xs_train], Y_train, batch_size=BATCH_SIZE, 
#           nb_epoch=NBR_EPOCHS, validation_data=([Xs_test, Xq_test, Xs_test], Y_test))

prediction.fit([train_data_matrix, train_options_matrix, train_data_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix, test_data_matrix], best_test_answer_matrix))
#Notes: dot_axis, concat _axis? Permute? 


#Doubts: binary_crossentropy instead of categorical crossentropy for loss function

/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 4s 54ms/step - loss: 2.3093 - acc: 0.0940 - val_loss: 2.2740 - val_acc: 0.5233
Epoch 2/5
80/80 [==============================] - 4s 44ms/step - loss: 2.2548 - acc: 0.4521 - val_loss: 2.1735 - val_acc: 0.6835
Epoch 3/5
80/80 [==============================] - 3s 41ms/step - loss: 2.1359 - acc: 0.7017 - val_loss: 1.9897 - val_acc: 0.8209
Epoch 4/5
80/80 [==============================] - 3s 40ms/step - loss: 1.9341 - acc: 0.8610 - val_loss: 1.7156 - val_acc: 0.9547
Epoch 5/5
80/80 [==============================] - 3s 41ms/step - loss: 1.6427 - acc: 0.9642 - val_loss: 1.3574 - val_acc: 0.9991


In [33]:
prediction.save('model1_small.h5')
prediction_data = prediction.predict([test_data_matrix, test_options_matrix,test_data_matrix])
print (prediction_data)
prediction_shape = prediction_data.shape
print(prediction_shape)
print (prediction_data[0][0])
for i in range(0,prediction_shape[0]-1):
    for j in range(0,prediction_shape[1]-1):
        print (prediction_data[i][j])
        print (prediction_data[i][j].argmax(axis=0))


[[[ 0.23554838  0.08886301  0.08095115 ...,  0.08411545  0.08630057
    0.08323871]
  [ 0.20253241  0.09680884  0.08550391 ...,  0.08672262  0.09170651
    0.08675683]
  [ 0.20145866  0.09478073  0.08562806 ...,  0.08732651  0.09101351
    0.08710591]
  ..., 
  [ 0.43429428  0.06862132  0.05670865 ...,  0.06189857  0.06461132
    0.06019523]
  [ 0.19603622  0.09449463  0.08636995 ...,  0.08811036  0.09128892
    0.08784661]
  [ 0.37099051  0.07291375  0.06400805 ...,  0.0694778   0.07064679
    0.06760145]]

 [[ 0.23636977  0.08909797  0.08096705 ...,  0.0836822   0.08668361
    0.08297224]
  [ 0.19666262  0.09464578  0.08526628 ...,  0.09032255  0.08836738
    0.08893541]
  [ 0.19839403  0.0936703   0.08550157 ...,  0.089191    0.08927165
    0.08823423]
  ..., 
  [ 0.43517873  0.0687153   0.05667652 ...,  0.06160148  0.06479146
    0.05999925]
  [ 0.19404747  0.09375521  0.08627948 ...,  0.08934207  0.09012467
    0.08859023]
  [ 0.37803367  0.07388791  0.06382748 ...,  0.06687863  0

0
[ 0.47339207  0.06326069  0.0522643   0.06350855  0.05682391  0.05328964
  0.06391902  0.05779134  0.05984816  0.05590231]
0
[ 0.35533157  0.07689591  0.06593105  0.07634591  0.06977459  0.06679568
  0.07572762  0.07071769  0.07328585  0.06919412]
0
[ 0.39569756  0.07288658  0.06116732  0.07238442  0.06502724  0.06208758
  0.0710958   0.06616638  0.06893133  0.0645558 ]
0
[ 0.47799367  0.0627226   0.05174858  0.06299542  0.0563214   0.05277661
  0.06344041  0.05728725  0.05932353  0.05539056]
0
[ 0.30661997  0.08356886  0.07188914  0.08207921  0.07456718  0.07258198
  0.07897047  0.0757869   0.07925938  0.07467688]
0
[ 0.20297322  0.09634917  0.08543926  0.09299399  0.08538782  0.08560229
  0.08623643  0.08676904  0.09150185  0.08674693]
0
[ 0.11649992  0.09920891  0.09835391  0.09689782  0.09832387  0.09794919
  0.09836781  0.09747875  0.09880304  0.09811674]
0
[ 0.30412075  0.08069189  0.07213136  0.08053301  0.07645052  0.07292909
  0.08281874  0.07675354  0.07831593  0.07525513]


0
[ 0.36285028  0.0767455   0.06496865  0.07578806  0.0684779   0.06580106
  0.07461717  0.06992187  0.07246624  0.06836332]
0
[ 0.26057267  0.08650994  0.0779509   0.08603454  0.08118557  0.07863429
  0.08408029  0.08066456  0.08444428  0.07992294]
0
[ 0.15630919  0.09813232  0.09103569  0.09332506  0.09072682  0.09063815
  0.09833831  0.0949943   0.09246352  0.09403663]
0
[ 0.39963374  0.07297549  0.06056165  0.07193089  0.06409565  0.06142126
  0.07094266  0.06598475  0.06820909  0.06424482]
0
[ 0.22968739  0.09089058  0.08120151  0.08792576  0.08291632  0.08144781
  0.08986577  0.08574244  0.08585719  0.0844653 ]
0
[ 0.38388169  0.07393981  0.06278691  0.07400106  0.06683394  0.06377308
  0.07129021  0.06695642  0.07090347  0.0656335 ]
0
[ 0.15523253  0.09719265  0.09180108  0.09401221  0.09227259  0.09161436
  0.0964596   0.09393305  0.09393204  0.0935499 ]
0
[ 0.30614525  0.08232014  0.07200728  0.08156185  0.07539857  0.07277016
  0.08004226  0.07593425  0.07906291  0.07475734]


0
[ 0.16988088  0.09697937  0.08905896  0.09235001  0.08910469  0.08877882
  0.09703564  0.09339822  0.09112275  0.09229068]
0
[ 0.14975511  0.09647889  0.09329147  0.09506571  0.09454504  0.09330758
  0.09476998  0.0932339   0.09601358  0.09353884]
0
[ 0.1871639   0.09428214  0.0872914   0.09174398  0.08865913  0.08739381
  0.09308954  0.0900349   0.09094561  0.08939558]
0
[ 0.21235585  0.09202557  0.08380133  0.08969034  0.08562193  0.0840569
  0.09066923  0.08717355  0.08831204  0.08629332]
0
[ 0.21317248  0.09144635  0.08397437  0.08992208  0.08620885  0.08434201
  0.08963653  0.08651182  0.08889323  0.0858923 ]
0
[ 0.1673844   0.09667815  0.08972253  0.09284289  0.09010959  0.08953162
  0.09628039  0.09309743  0.09205374  0.0922992 ]
0
[ 0.23212594  0.09084293  0.08077405  0.08760306  0.08238199  0.08099414
  0.08997179  0.08567082  0.08535866  0.08427665]
0
[ 0.34135681  0.07919615  0.06743594  0.07781815  0.07067769  0.06818399
  0.07727685  0.07253871  0.07456492  0.07095077]
0

  0.08758812  0.08445697  0.08570359  0.08274686]
0
[ 0.14806746  0.10237013  0.0920941   0.09827081  0.09240787  0.09225723
  0.09515934  0.09393764  0.09327284  0.0921626 ]
0
[ 0.11489309  0.09874279  0.09877868  0.09662409  0.09871545  0.09831854
  0.09845009  0.09762721  0.09929721  0.09855286]
0
[ 0.18275854  0.09632104  0.08771689  0.09374755  0.08916822  0.08799937
  0.09232467  0.08997305  0.09110522  0.08888544]
0
[ 0.18872249  0.09812061  0.08634517  0.09469973  0.08766239  0.08676724
  0.09154924  0.08931549  0.08945111  0.08736651]
0
[ 0.36369842  0.07520533  0.06523028  0.07496237  0.06900477  0.06603982
  0.07439667  0.06966866  0.07316247  0.06863118]
0
[ 0.19916113  0.09629922  0.0850886   0.09330487  0.08669154  0.08553442
  0.09064975  0.08813029  0.08875282  0.08638734]
0
[ 0.39119554  0.06891147  0.06272313  0.06999457  0.06701545  0.06343721
  0.07177905  0.06657141  0.07172599  0.06664619]
0
[ 0.19293547  0.09836037  0.08561503  0.09473356  0.08694635  0.08608885


...

0
[ 0.15044887  0.09848286  0.09358906  0.09363544  0.09342489  0.09631506
  0.09384013  0.09463096  0.09379925  0.09183342]
0
[ 0.11466577  0.09920821  0.09858146  0.09719359  0.0985805   0.09779961
  0.09861529  0.09753656  0.09922595  0.09859302]
0
[ 0.18395574  0.09473354  0.08829702  0.09182822  0.08956216  0.08959765
  0.09175976  0.09023319  0.0913019   0.08873077]
0
[ 0.19126981  0.09481934  0.08752187  0.09075084  0.08845869  0.09003863
  0.09039776  0.08984515  0.08984639  0.08705152]
0
[ 0.36223516  0.07623675  0.06496475  0.07622655  0.06883841  0.06520128
  0.07481661  0.06958136  0.07311152  0.06878757]
0
[ 0.20131262  0.09365412  0.08601673  0.0901287   0.08731866  0.08812787
  0.08971402  0.08854297  0.08905984  0.08612457]
0
[ 0.38551599  0.07260927  0.06180135  0.07462248  0.06644857  0.0604779
  0.07337433  0.06629909  0.07158358  0.06726737]
0
[ 0.19589131  0.09460054  0.08693938  0.09024909  0.08784119  0.08978616
  0.08978746  0.08943615  0.08918352  0.08628517]
0

  0.06621422  0.06114938  0.06533937  0.05823481]
0
[ 0.16028184  0.09547384  0.09206031  0.09420553  0.09372199  0.09089024
  0.09452379  0.09232297  0.09485343  0.0916661 ]
0
[ 0.26512587  0.08554481  0.07808134  0.08553614  0.08181223  0.07665482
  0.08443242  0.08039764  0.08409238  0.07832231]
0
[ 0.13992749  0.09858058  0.09356903  0.09530908  0.09346128  0.095512
  0.096414    0.09507228  0.09574242  0.09641186]
0
[ 0.35819682  0.07618863  0.0664498   0.07653029  0.07079051  0.06553683
  0.07478037  0.07003581  0.07385357  0.06763736]
0
[ 0.24381202  0.08883596  0.07963529  0.08721091  0.08210016  0.08064137
  0.08666399  0.08314134  0.08543216  0.08252678]
0
[ 0.15834554  0.0972743   0.09063493  0.09400508  0.09090833  0.0929993
  0.09481867  0.09301875  0.09376116  0.09423404]
0
[ 0.13470979  0.09921801  0.09412006  0.09559618  0.09366099  0.09656671
  0.09687182  0.09573647  0.09607136  0.09744854]
0
[ 0.31948322  0.07865828  0.07262197  0.08066071  0.07806669  0.06854703
  0

0
[ 0.12446792  0.09893531  0.09711047  0.09618742  0.09689086  0.09694026
  0.09762833  0.09676474  0.09783759  0.09723705]
0
[ 0.31456384  0.08000202  0.07034797  0.08320426  0.07697     0.07000076
  0.07941887  0.07342745  0.07966056  0.0724043 ]
0
[ 0.22900952  0.09037177  0.08196152  0.08796176  0.08363052  0.08268715
  0.0881036   0.08515587  0.08678517  0.08433305]
0
[ 0.301085    0.08331553  0.07266667  0.08142699  0.07521892  0.073623
  0.08083863  0.07702583  0.07905982  0.07573962]
0
[ 0.35948804  0.07582879  0.06507338  0.07799552  0.07093723  0.06516547
  0.07469875  0.06894277  0.07420886  0.06766121]
0
[ 0.43391177  0.06825575  0.0565289   0.0696451   0.06175655  0.0570106
  0.06667543  0.06111929  0.06551598  0.05958062]
0
[ 0.12616754  0.09953126  0.09708513  0.09494738  0.09533712  0.09750032
  0.09747493  0.09739104  0.09685092  0.09771437]
0
[ 0.44597208  0.06765248  0.05538241  0.06723724  0.05923171  0.05634516
  0.06536537  0.0605334   0.06342945  0.05885066]
0
[

[ 0.4780767   0.06098757  0.05076514  0.06224477  0.05719731  0.05402146
  0.06327818  0.05514067  0.06172122  0.05656705]
0
[ 0.19235216  0.09692367  0.08814143  0.0933499   0.08684333  0.08550239
  0.08999659  0.09110431  0.08893459  0.08685164]
0
[ 0.32421505  0.08086532  0.06978438  0.0798355   0.0729848   0.0703808
  0.07832418  0.07422255  0.07675121  0.07263616]
0
[ 0.19446228  0.09661363  0.08779985  0.0931216   0.08665138  0.08527557
  0.0898537   0.09078813  0.08878649  0.08664735]
0
[ 0.44681031  0.06525987  0.05454503  0.06601445  0.06029662  0.05720122
  0.06634229  0.05907991  0.06471999  0.05973033]
0
[ 0.23308262  0.08727035  0.08044704  0.08706518  0.08504256  0.08278647
  0.08877894  0.08281983  0.08818818  0.08451889]
0
[ 0.13918895  0.10348654  0.09684312  0.09829132  0.09227718  0.0919045
  0.09380405  0.09854314  0.09312169  0.09253953]
0
[ 0.33647957  0.07564391  0.06693284  0.0766774   0.07361328  0.07060366
  0.07886817  0.07044413  0.07781061  0.07292646]
0
[ 

0
[ 0.41147298  0.07465822  0.06084235  0.06916904  0.06193857  0.06167123
  0.06469728  0.06193806  0.06981481  0.06379747]
0
[ 0.15048452  0.0958083   0.09241291  0.09532285  0.09432564  0.09239519
  0.09690595  0.0944275   0.09459949  0.09331767]
0
[ 0.18423437  0.09401125  0.087997    0.09228444  0.08972715  0.08819149
  0.09239756  0.08978491  0.09192905  0.08944286]
0
[ 0.39315167  0.07667571  0.06300603  0.07104644  0.06399308  0.06380842
  0.06660538  0.06392682  0.07188921  0.06589717]
0
[ 0.31240487  0.08332483  0.07188714  0.08008183  0.07391434  0.0725838
  0.07756259  0.07423817  0.07938126  0.07462113]
0
[ 0.41508171  0.07306016  0.05987179  0.06942926  0.06197715  0.06074376
  0.0659248   0.06243646  0.06846887  0.06300601]
0
[ 0.33211535  0.07976515  0.06871774  0.07896708  0.07221201  0.06951757
  0.07767224  0.07323142  0.07599268  0.07180873]
0
[ 0.36636603  0.08048897  0.06662462  0.07327308  0.06667012  0.06734915
  0.06818177  0.06616415  0.07561682  0.06926527]
0

In [34]:
print(train_data_matrix.shape)
print(train_options_matrix.shape)
print(train_answer_matrix.shape)

(80, 729)
(80, 729)
(80, 729)


In [35]:
from keras.layers import Dense, Merge, Dropout, Permute, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import RNN
from keras.models import Sequential

# Contants
VOCABULARY_SIZE = 729 #53628 ## estimate from the goldilocks paper
HIDDEN_LAYER_SIZE = 10  ## 10 neurons for 10 options
BATCH_SIZE = 20 #32
EPOCHS_SIZE = 10

#Initial Embedding matrices A, B for story and options
#Each sentence is taken as an n-dimensional vector of float64, with n=VOCABULARY-SIZE
#memory_encoder encodes lines 1-21 from story, answer encoder
memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length=729)) #21
#memory_encoder.add(RNN(cell, input_dim=HIDDEN_LAYER_SIZE, input_lenght=21, return_sequences=False) )
memory_encoder.add(Dropout(0.3))

answer_encoder = Sequential()
answer_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length=729)) #1
answer_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
prediction = Sequential()
prediction.add(Merge([memory_encoder, answer_encoder], mode ='dot'))
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation('softmax')) #for softmax 

#Optimizer, loss function, metrics setting
prediction.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#training
prediction.fit([train_data_matrix, train_options_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix], best_test_answer_matrix))
  
#scores = prediction.evaluate([test_data_matrix, test_options_matrix], best_test_answer_matrix)
#print("%s: %.2f%%" % (prediction.metrics_names[1], scores[1]*100))

/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 1s 17ms/step - loss: 2.2735 - acc: 0.6722 - val_loss: 2.2318 - val_acc: 0.9997
Epoch 2/10
80/80 [==============================] - 1s 12ms/step - loss: 2.2123 - acc: 0.9714 - val_loss: 2.1461 - val_acc: 0.9997
Epoch 3/10
80/80 [==============================] - 1s 12ms/step - loss: 2.1230 - acc: 0.9910 - val_loss: 2.0282 - val_acc: 0.9997
Epoch 4/10
80/80 [==============================] - 1s 12ms/step - loss: 2.0016 - acc: 0.9968 - val_loss: 1.8711 - val_acc: 0.9997
Epoch 5/10
80/80 [==============================] - 1s 12ms/step - loss: 1.8377 - acc: 0.9983 - val_loss: 1.6677 - val_acc: 0.9997
Epoch 6/10
80/80 [==============================] - 1s 12ms/step - loss: 1.6289 - acc: 0.9985 - val_loss: 1.4144 - val_acc: 0.9997
Epoch 7/10
80/80 [==============================] - 1s 12ms/step - loss: 1.3697 - acc: 0.9986 - val_loss: 1.1170 - val_acc: 0.9997
Epoch 8/10
80/80 [=====================

In [36]:
prediction.save('model2_small.h5')

In [37]:
best_train_answer_matrix = np.expand_dims(train_answer_matrix, -1)
best_test_answer_matrix = np.expand_dims(test_answer_matrix, -1)

In [38]:
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [39]:
import numpy as np
#np.set_printoptions(threshold=30)

splitted_train_data_matrix=np.vsplit(train_data_matrix,80)
splitted_train_options_matrix=np.vsplit(train_options_matrix,80)
splitted_train_answer_matrix=np.vsplit(train_answer_matrix,80)
#print(splitted_train_data_matrix[0])
print(splitted_train_data_matrix[0].shape)
print(splitted_train_options_matrix[0].shape)
print(splitted_train_answer_matrix[0].shape)
print("**")
print(splitted_train_answer_matrix[0].shape[1])


(1, 729)
(1, 729)
(1, 729)
**
729


In [40]:
D3_splitted_train_data_matrix=np.asarray(splitted_train_data_matrix)
print(D3_splitted_train_data_matrix.shape)
D3_splitted_train_options_matrix=np.asarray(splitted_train_options_matrix)
print(D3_splitted_train_options_matrix.shape)
D3_splitted_train_answer_matrix=np.asarray(splitted_train_answer_matrix)
print(D3_splitted_train_answer_matrix.shape)

(80, 1, 729)
(80, 1, 729)
(80, 1, 729)


In [41]:
#Load Keras models

model1 = load_model('model1.h5')
model2 = load_model('model2.h5')
model1.evaluate(competition_data_matrix,competition_options_matrix)
model2.evaluate(competition_data_matrix,competition_options_matrix)

NameError: name 'load_model' is not defined